In [1]:
class Graph(object):
    def __init__(self, cost_matrix: list, size: int):
        self.matrix = cost_matrix
        self.rank = size

        # noinspection PyUnusedLocal
        #initialize the pheromone value: the pheromone on paths between each pair of city is initialized as the same as "1 / (size * size)"
        self.pheromone = [
            [
                1 / (size * size)
                for j in range(size)
            ]
            for i in range(size)
        ]

        # print(self.pheromone)


In [2]:

class ACO(object):
    def __init__(self, ant_count: int, generations: int, alpha: float, beta: float, rho: float, q: int,
                 strategy: int):
        """
        :param ant_count:
        :param generations:
        :param alpha: relative importance of pheromone
        :param beta: relative importance of heuristic information
        :param rho: pheromone residual coefficient
        :param q: pheromone intensity
        :param strategy: pheromone update strategy. 0 - ant-cycle, 1 - ant-quality, 2 - ant-density
        """
        self.Q = q
        self.rho = rho
        self.beta = beta
        self.alpha = alpha
        self.ant_count = ant_count
        self.generations = generations
        self.update_strategy = strategy

    def _update_pheromone(self, graph: Graph, ants: list):
        for i, row in enumerate(graph.pheromone):
            for j, col in enumerate(row):
                graph.pheromone[i][j] *= self.rho
                for ant in ants:
                    graph.pheromone[i][j] += ant.pheromone_delta[i][j]

    # noinspection PyProtectedMember
    def solve(self, graph: Graph):
        best_cost = float('inf')
        best_solution = []

        for gen in range(self.generations):
            # noinspection PyUnusedLocal
            # now we generate ant with same initliazed attributes and quantity of ant_count
            ants = [_Ant(self, graph) for i in range(self.ant_count)]

            for ant in ants:
                for i in range(graph.rank - 1):
                    ant._select_next()
                ant.total_cost += graph.matrix[ant.tabu[-1]][ant.tabu[0]]
                if ant.total_cost < best_cost:
                    best_cost = ant.total_cost
                    best_solution = [] + ant.tabu
                # update pheromone
                ant._update_pheromone_delta()
            self._update_pheromone(graph, ants)
            # print(best_solution)
            print(best_cost)
            # print('generation #{}, best cost: {}, path: {}'.format(gen, best_cost, best_solution))
            
        return best_solution, best_cost


In [3]:
import random

class _Ant(object):
    def __init__(self, aco: ACO, graph: Graph):
        self.colony = aco
        self.graph = graph
        self.total_cost = 0.0
        self.tabu = []  # tabu list
        self.pheromone_delta = []  # the local increase of pheromone
        self.allowed = [i for i in range(graph.rank)]  # numbernodes which are allowed for the next selection
        # print([i for i in range(graph.rank)])
        # print(graph.rank)
        self.eta = [[0 if i == j else 1 / graph.matrix[i][j] for j in range(graph.rank)] for i in
                    range(graph.rank)]  # heuristic information
        # print(self.eta)
        start = random.randint(0, graph.rank - 1)  # start from any node
        self.tabu.append(start)
        self.current = start
        self.allowed.remove(start)

    def _select_next(self):
        denominator = 0
        for i in self.allowed:
            denominator += self.graph.pheromone[self.current][i] ** self.colony.alpha * self.eta[self.current][
                                                                                            i] ** self.colony.beta
        # noinspection PyUnusedLocal
        probabilities = [0 for i in range(self.graph.rank)]  # probabilities for moving to a node in the next step
        for i in range(self.graph.rank):
            try:
                self.allowed.index(i)  # test if allowed list contains i
                probabilities[i] = self.graph.pheromone[self.current][i] ** self.colony.alpha * \
                    self.eta[self.current][i] ** self.colony.beta / denominator
            except ValueError:
                pass  # do nothing
        # select next node by probability roulette
        selected = 0
        rand = random.random()
        for i, probability in enumerate(probabilities):
            rand -= probability
            if rand <= 0:
                selected = i
                break
        self.allowed.remove(selected)
        self.tabu.append(selected)
        self.total_cost += self.graph.matrix[self.current][selected]
        self.current = selected

    # noinspection PyUnusedLocal
    def _update_pheromone_delta(self):
        self.pheromone_delta = [[0 for j in range(self.graph.rank)] for i in range(self.graph.rank)]
        for _ in range(1, len(self.tabu)):
            i = self.tabu[_ - 1]
            j = self.tabu[_]
            if self.colony.update_strategy == 1:  # ant-quality system
                self.pheromone_delta[i][j] = self.colony.Q
            elif self.colony.update_strategy == 2:  # ant-density system
                # noinspection PyTypeChecker
                self.pheromone_delta[i][j] = self.colony.Q / self.graph.matrix[i][j]
            else:  # ant-cycle system
                self.pheromone_delta[i][j] = self.colony.Q / self.total_cost

In [4]:
import math

# from aco import ACO, Graph
# from plot import plot

# used to calculate the distance between two cities
def distance(city1: dict, city2: dict):
    return math.sqrt((city1['x'] - city2['x']) ** 2 + (city1['y'] - city2['y']) ** 2)


def main():
    cities = []
    points = []
    index_count = 0

    # read cities from the dataset
    with open('./data_100.txt') as file:
        for line in file.readlines():
            city = line.split(' ')
            cities.append(dict(index = index_count, x=int(city[0]), y=int(city[1])))
            points.append((int(city[0]), int(city[1])))
            index_count += 1

    cost_matrix = []
    size = len(cities)
    # print(size)

    # now we have the adjency list for the cities
    for i in range(size):
        row = []
        for j in range(size):
            row.append(distance(cities[i], cities[j]))
        cost_matrix.append(row)

    # initialize the ACO attributes
    # ACO(ant_count: int, generations: int, alpha: float, beta: float, rho: float, q: int, strategy: int)
    aco = ACO(20, 50, 1.0, 10.0, 0.5, 10, 2)

    # initialize the Graph with all same 
    # Graph(cost_matrix: list, size: int)
    graph = Graph(cost_matrix, size)

    # run ACO on Graph
    path, cost = aco.solve(graph)
    print('final cost: {}, final path: {}'.format(cost, path))

if __name__ == '__main__':
    main()


46408.07443734277
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44183.47950669027
44175.78230028193
44175.78230028193
final cost: 44175.78230028193, final path: [47, 70, 18, 6, 39, 46, 54, 23, 10, 9, 60, 63, 45, 22, 49